# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [25]:
import graphlab
import numpy

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('../data/kc_house_data.gl/')
print "sales.shape = ", sales.shape
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create for academic use is assigned to bmatthewtaylor@gmail.com and will expire on October 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\username\AppData\Local\Temp\graphlab_server_1483760957.log.0


sales.shape =  (21613, 21)


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 
from math import *
print "numpy loaded"

numpy loaded


In [4]:
def get_numpy_data(data_sframe, features, output):
    #copied from week-2-multiple-regression-assignment-2-gradient_descent_working_A 
    data_sframe['constant'] = 1 
    # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can 
    # extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the 
    # SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)
print "get_numpy_data loaded"

get_numpy_data loaded


Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    #copied from week-2-multiple-regression-assignment-2-gradient_descent_working_A
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    #print "predict_output called, performing dot mult"
    predictions = np.dot(feature_matrix, weights)
    #print "predict_output called, dot mult completed"
    return(predictions)
print "predict_output loaded"

predict_output loaded


# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms
#https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html
#numpy.linalg.norm(x, ord=None, axis=None, keepdims=False)
#This function is able to return one of eight different matrix norms, or one of an infinite number of vector norms 
#(described below), depending on the value of the ord parameter.
#ord : {non-zero int, inf, -inf, ‘fro’, ‘nuc’}, optional
#ord	norm for matrices	norm for vectors
#None	Frobenius norm	   2-norm
# Order of the norm (see table under Notes). inf means numpy’s inf object.
#n : float or ndarray : Norm of the matrix or vector(s).

#NB: default when ord is not not nominated is Frobenius norm for matrices
#http://mathworld.wolfram.com/FrobeniusNorm.html

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms 
# gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [56]:
#verbose version
#NB: messy. debugging identified wrong output object type
#NBB: functional description above does not nominate output object types requ'd.
def normalize_features(feature_matrix):
    #normalizes columns of a given feature matrix.
    normalized_features = feature_matrix
    norms = []
    num_cols = feature_matrix.shape[1]
    #print "num_cols = ", num_cols
    for i in range(num_cols):
        #print "normalizing column ", i
        X = feature_matrix[:,i]
        norms.append(np.linalg.norm(X, axis=0))
        normalized_features[:,i] = X/norms[i]
    norms = np.array(norms)
    return normalized_features, norms
print "normalize_features defined"

normalize_features defined


In [58]:
import numpy
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print "type(features) = ", type(features), ", features.shape = ", features.shape
print "type(norms) =", type(norms), ", norms.shape = ", norms.shape
print "---------------\nfeatures = \n", features
if numpy.array_equal(features, np.array( [[ 0.6, 0.6, 0.6], [ 0.8, 0.8, 0.8]])):
    print "array matches, is ok"
else:
    print "error"
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print "---------------\nnorms = \n", norms
# should print
# [5.  10.  15.]
if numpy.array_equal(norms, np.array([5.0, 10.0, 15.0])):
    print "norms match, is ok"
else:
    print "error"

type(features) =  <type 'numpy.ndarray'> , features.shape =  (2L, 3L)
type(norms) = <type 'numpy.ndarray'> , norms.shape =  (3L,)
---------------
features = 
[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
array matches, is ok
---------------
norms = 
[  5.  10.  15.]
norms match, is ok


In [59]:
#compact version
def normalize_features(feature_matrix):
    #receives numpy.ndarray, calculates norm for each column,
    #normalises each column, 
    #returns normalised numpy.ndarray [same # rows & columns as input] and 
    # numpy.ndarray [1D array] of norms.
    #print "type(feature_matrix) = ", type(feature_matrix)
    #print "feature_matrix.shape = ", feature_matrix.shape
    norms = np.linalg.norm(feature_matrix, axis=0)
    features=feature_matrix/norms
    return features,norms

To test the function, run the following:

In [60]:
import numpy
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print "type(features) = ", type(features), ", features.shape = ", features.shape
print "type(norms) =", type(norms), ", norms.shape = ", norms.shape
print "---------------\nfeatures = \n", features
if numpy.array_equal(features, np.array( [[ 0.6, 0.6, 0.6], [ 0.8, 0.8, 0.8]])):
    print "array matches, is ok"
else:
    print "error"
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print "---------------\nnorms = \n", norms
# should print
# [5.  10.  15.]
if numpy.array_equal(norms, np.array([5.0, 10.0, 15.0])):
    print "norms match, is ok"
else:
    print "error"

type(features) =  <type 'numpy.ndarray'> , features.shape =  (2L, 3L)
type(norms) = <type 'numpy.ndarray'> , norms.shape =  (3L,)
---------------
features = 
[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
array matches, is ok
---------------
norms = 
[  5.  10.  15.]
norms match, is ok


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [70]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
print type(simple_feature_matrix), simple_feature_matrix.shape
print type(output), output.shape

<type 'numpy.ndarray'> (21613L, 3L)
<type 'numpy.ndarray'> (21613L,)


Don't forget to normalize features:

In [71]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
print "simple_feature_matrix.shape = ", simple_feature_matrix.shape
print type(norms), len(norms)

simple_feature_matrix.shape =  (21613L, 3L)
<type 'numpy.ndarray'> 3


We assign some random set of initial weights and inspect the values of `ro[i]`:

In [72]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [73]:
#predict_output(feature_matrix, weights)
prediction = predict_output(simple_feature_matrix, weights)
print type(prediction), prediction.shape

<type 'numpy.ndarray'> (21613L,)


Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [74]:
print "simple_feature_matrix.shape = ", simple_feature_matrix.shape
num_features = int(simple_feature_matrix.shape[1]-1)#number of columns
print "num_features (excluding intercept) = ", num_features
print "type(num_features) = ", type(num_features)
ro = []
print "ro = ", ro
print "type(ro) = ", type(ro)
#for i in range(1, len(simple_features)+1):
for i in range(len(simple_features)+1):
    #loop over all columns
    #ro.append(SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ])
    feature_i = simple_feature_matrix[:,i]
    #slice a column & calculate 
    temp =  (feature_i*(output - prediction + weights[i]*feature_i) ).sum()
    ro.append(temp)
print "ro = ", ro

simple_feature_matrix.shape =  (21613L, 3L)
num_features (excluding intercept) =  2
type(num_features) =  <type 'int'>
ro =  []
type(ro) =  <type 'list'>
ro =  [79400300.034929156, 87939470.772991076, 80966698.675965652]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [89]:
#take one step of coordinate descent on either feature 1 or feature 2.
#whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, 
#the corresponding weight w[i] is sent to zero.
from decimal import Decimal
print "for ro[0] = %.2E , l1_penalty must be < %.2E to cause w[1] to NOT be set to zero" % (Decimal(ro[0]), Decimal(2*ro[0]))
print "for ro[0] = %.2E, l1_penalty must be > %.2E to cause w[1] to be set to zero" % (Decimal(ro[0]), Decimal(2*ro[0]))
print "for ro[1] = %.2E, l1_penalty must be > %.2E to cause w[2] to be set to zero" % (Decimal(ro[1]), Decimal(2*ro[1]))
#NBB: variations in quiz question wording between multiple quiz attempts were observed. YMMV
#nb: need to recheck my formula used to define range of l1_penalty, 
# have sighted alternative formula.

for ro[0] = 7.94E+07 , l1_penalty must be < 1.59E+08 to cause w[1] to NOT be set to zero
for ro[0] = 7.94E+07, l1_penalty must be > 1.59E+08 to cause w[1] to be set to zero
for ro[1] = 8.79E+07, l1_penalty must be > 1.76E+08 to cause w[2] to be set to zero


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [93]:
#quiz questions 1 & 2
print 'when %.2E < l1_penalty < %.2E, w[1] will NOT be zero and w[2] will be zero' \
 % (Decimal(2*ro[0]), Decimal(2*ro[1]))
print 'when l1_penalty is > %.2E, **both** w[1] and w[2] will be set to zero' % Decimal(2*ro[0])

when 1.59E+08 < l1_penalty < 1.76E+08, w[1] will NOT be zero and w[2] will be zero
when l1_penalty is > 1.59E+08, **both** w[1] and w[2] will be set to zero


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

[formula/logic repeated from above]

The formula for optimizing each coordinate is as follows:
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2

In [91]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:,i] # pull this out for readability.
    ro_i = (feature_i*(output - prediction + weights[i]*feature_i) ).sum()
    #or rewrite above two lines as one line.
    #ro_i = (feature_matrix[:,i]*(output-prediction+weights[i]*feature_matrix[:,i])).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [92]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [131]:
def get_RSS(pred, outcome):
    #rss = residual sum of squares = (pred - outcome)^2
    #helper method to aid readability
    res= outcome-pred
    rss=(res*res).sum()
    return(rss)   

In [132]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    d=2*tolerance
    while  (d >tolerance):
        weights=initial_weights
        deltas= [0 for i in range(len(weights))]
        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

            # use old_weights_i to compute change in coordinate
            deltas[i]=abs(weights[i]-old_weights_i)
        d= max(deltas)
      #  print d
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [133]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [134]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) 
# normalize features
print "simple_feature_matrix = ", type(simple_feature_matrix), simple_feature_matrix.shape
print "output = ", type(output), output.shape
print "normalized_simple_feature_matrix = ", type(normalized_simple_feature_matrix), normalized_simple_feature_matrix.shape
print "simple_norms = ",  type(simple_norms), simple_norms.shape

simple_feature_matrix =  <type 'numpy.ndarray'> (21613L, 3L)
output =  <type 'numpy.ndarray'> (21613L,)
normalized_simple_feature_matrix =  <type 'numpy.ndarray'> (21613L, 3L)
simple_norms =  <type 'numpy.ndarray'> (3L,)


Then, run your implementation of LASSO coordinate descent:

In [135]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, 
                                            output,
                                            initial_weights, 
                                            l1_penalty, tolerance)
print "weights = ", weights, weights.shape
print "type(normalized_simple_feature_matrix) = ", type(normalized_simple_feature_matrix)
print "type(weights) = ", type(weights)
prediction = predict_output(normalized_simple_feature_matrix, weights)
RSS = ((prediction-output)**2).sum()
#quiz Q3 answer
print "RSS = ", RSS
print 'Quiz Q3 answer: RSS = %.2E' % Decimal(RSS)
# recall: normalized_simple_feature_matrix is generated from 
# simple_features = ['sqft_living', 'bedrooms']
# weights represents ['intercept', 'sqft_living', 'bedrooms'] 
# ('intercept' also referred to as 'constant')
# 
temp = ['constant']+simple_features
print "quiz Q 4 answer: features & values"
for i  in range(len(temp)):
    print temp[i], " = %.2E" % Decimal(weights[i])
    

weights =  [ 21624998.36636292  63157246.78545421         0.        ] (3L,)
type(normalized_simple_feature_matrix) =  <type 'numpy.ndarray'>
type(weights) =  <type 'numpy.ndarray'>
RSS =  1.63049248148e+15
Quiz Q3 answer: RSS = 1.63E+15
quiz Q 4 answer: features & values
constant  = 2.16E+07
sqft_living  = 6.32E+07
bedrooms  = 0.00E+00


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [136]:
train_data,test_data = sales.random_split(.8,seed=0)
print "train_data : ", type(train_data), train_data.shape
print "test_data : ", type(test_data), test_data.shape


train_data :  <class 'graphlab.data_structures.sframe.SFrame'> (17384, 22)
test_data :  <class 'graphlab.data_structures.sframe.SFrame'> (4229, 22)


Let us consider the following set of features.

In [137]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']
print "all_features : ", type(all_features), len(all_features)

all_features :  <type 'list'> 13


First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [138]:
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
norm_feature_matrix, norms = normalize_features(feature_matrix)
print norm_feature_matrix
print "feature_matrix : ", type(feature_matrix), feature_matrix.shape
print "output : ", type(output), output.shape
print "norm_feature_matrix : ", type(norm_feature_matrix), norm_feature_matrix.shape
print "norms : ", type(norms), norms.shape
#NB: all_features has 13 element, norms has 14 elements, first element is for constant.

[[ 0.00758447  0.00652117  0.0033687  ...,  0.          0.00752148  0.        ]
 [ 0.00758447  0.00652117  0.00757957 ...,  0.0057043   0.0075061
   0.03707954]
 [ 0.00758447  0.00434745  0.0033687  ...,  0.          0.00743684  0.        ]
 ..., 
 [ 0.00758447  0.00652117  0.00842175 ...,  0.          0.00772924  0.        ]
 [ 0.00758447  0.00652117  0.00842175 ...,  0.          0.00771     0.        ]
 [ 0.00758447  0.00434745  0.00252652 ...,  0.          0.00772539  0.        ]]
feature_matrix :  <type 'numpy.ndarray'> (17384L, 14L)
output :  <type 'numpy.ndarray'> (17384L,)
norm_feature_matrix :  <type 'numpy.ndarray'> (17384L, 14L)
norms :  <type 'numpy.ndarray'> (14L,)


In [152]:
my_output = 'price'
initial_weights = np.zeros(14)#Initialize weights to all zeros,
l1_penalty = 1e7
tolerance = 1
weights1e7 = lasso_cyclical_coordinate_descent(norm_feature_matrix, 
                                               output,
                                               initial_weights, 
                                               l1_penalty, tolerance)
print "weights1e7 : ", type(weights1e7), weights1e7.shape
predictions=predict_output(norm_feature_matrix, weights1e7)
rss= get_RSS(predictions, output)
print "rss : ", type(rss), " = ", rss
features = ['constant']+all_features
print "quiz Q5 answer"
for i in range(len(features)):
    #print features[i], " = ", weights1e7[i]
    print " %15s = %.2E" % (features[i], Decimal(weights1e7[i]))

weights1e7 :  <type 'numpy.ndarray'> (14L,)
rss :  <type 'numpy.float64'>  =  1.23159557516e+15
quiz Q5 answer
        constant = 2.44E+07
        bedrooms = 0.00E+00
       bathrooms = 0.00E+00
     sqft_living = 4.84E+07
        sqft_lot = 0.00E+00
          floors = 0.00E+00
      waterfront = 3.32E+06
            view = 7.33E+06
       condition = 0.00E+00
           grade = 0.00E+00
      sqft_above = 0.00E+00
   sqft_basement = 0.00E+00
        yr_built = 0.00E+00
    yr_renovated = 0.00E+00


First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [151]:
my_output = 'price'
initial_weights = np.zeros(14)
l1_penalty = 1e8
tolerance = 1
weights1e8 = lasso_cyclical_coordinate_descent(norm_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
predictions=predict_output(norm_feature_matrix, weights1e8)
rss= get_RSS(predictions, output)
print "rss : ", type(rss), " = ", rss
features = ['constant']+all_features
print "quiz Q6 answer, weights"
for i in range(len(features)):
    #print features[i], " = ", weights1e7[i]
    print " %15s = %.2E" % (features[i], Decimal(weights1e8[i]))

rss :  <type 'numpy.float64'>  =  2.37576185506e+15
quiz Q6 answer, weights
        constant = 7.11E+07
        bedrooms = 0.00E+00
       bathrooms = 0.00E+00
     sqft_living = 0.00E+00
        sqft_lot = 0.00E+00
          floors = 0.00E+00
      waterfront = 0.00E+00
            view = 0.00E+00
       condition = 0.00E+00
           grade = 0.00E+00
      sqft_above = 0.00E+00
   sqft_basement = 0.00E+00
        yr_built = 0.00E+00
    yr_renovated = 0.00E+00


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [155]:
my_output = 'price'
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5
weights1e4= lasso_cyclical_coordinate_descent(norm_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
#print weights1e4
predictions=predict_output(norm_feature_matrix, weights1e4)
rss= get_RSS(predictions, output)
#print rss

print "rss : ", type(rss), " = ", rss
features = ['constant']+all_features
print "quiz Q7 answer, weights"
for i in range(len(features)):
    #print features[i], " = ", weights1e7[i]
    print " %15s = %.2E" % (features[i], Decimal(weights1e4[i]))

rss :  <type 'numpy.float64'>  =  9.84789065063e+14
quiz Q7 answer, weights
        constant = 7.78E+07
        bedrooms = -2.29E+07
       bathrooms = 1.53E+07
     sqft_living = 9.22E+07
        sqft_lot = -2.14E+06
          floors = -8.82E+06
      waterfront = 6.49E+06
            view = 7.07E+06
       condition = 4.12E+06
           grade = 1.84E+07
      sqft_above = -1.46E+07
   sqft_basement = -5.53E+06
        yr_built = -8.36E+07
    yr_renovated = 2.78E+06


***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [157]:
normalized_weights1e4 = weights1e4/norms
normalized_weights1e7 = weights1e7/norms
normalized_weights1e8 = weights1e8/norms
print "normalized_weights1e4 : ", type(normalized_weights1e4), normalized_weights1e4.shape
print "normalized_weights1e7 : ", type(normalized_weights1e7), normalized_weights1e7.shape
print "normalized_weights1e8 : ", type(normalized_weights1e8), normalized_weights1e8.shape
print normalized_weights1e7[3]

normalized_weights1e4 :  <type 'numpy.ndarray'> (14L,)
normalized_weights1e7 :  <type 'numpy.ndarray'> (14L,)
normalized_weights1e8 :  <type 'numpy.ndarray'> (14L,)
161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [158]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [160]:
predictions=predict_output(test_feature_matrix, normalized_weights1e4)
rss= get_RSS(predictions, test_output)
print "rss for normalized_weights1e4 = ", rss

rss for normalized_weights1e4 =  2.2778100476e+14


In [161]:
predictions=predict_output(test_feature_matrix, normalized_weights1e7)
rss= get_RSS(predictions, test_output)
print "rss for normalized_weights1e7 = ", rss

rss for normalized_weights1e7 =  2.75962079909e+14


In [162]:
predictions=predict_output(test_feature_matrix, normalized_weights1e8)
rss= get_RSS(predictions, test_output)
print "rss for normalized_weights1e8 = ", rss

rss for normalized_weights1e8 =  5.37166150034e+14


***QUIZ QUESTION***

Which model performed best on the test data?